<a href="https://colab.research.google.com/github/googspell/pproject4/blob/main/Analys_with_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Генерация датасета и загрузка базы


In [ ]:
#техническая штука для работы sqlite3 оконных функций
! gdown 1CDqqyKV9BvrvnuR1l8HVz0aYuM4yOTUr
! mv _sqlite3.cpython-38-x86_64-linux-gnu.so /usr/lib/python3.8/lib-dynload/
import os
os.kill(os.getpid(), 9)

Downloading...
From: https://drive.google.com/uc?id=1CDqqyKV9BvrvnuR1l8HVz0aYuM4yOTUr
To: /content/_sqlite3.cpython-38-x86_64-linux-gnu.so
100% 6.59M/6.59M [00:00<00:00, 98.8MB/s]


In [1]:
#импортируем библиотеки
import numpy as np
import pandas as pd
import random
import sqlite3
import datetime

In [2]:
#создадим датафрейм
#user_id-идентфикатор пользователя
#order_id-номер заказа
#click2delivery-время с момента оформления заказа
#order_item_sum-общая стоимость заказа
#retention -день жизни покупателя в который он совершил заказ
#order_item_profit-прибыль от заказа

#фиксация_времени
np.random.seed(42)
random.seed(42)

#функция гереации id из символов
def generate_user_id():
  charrete = 'abcdefj123456789'
  return ''.join(random.choice(charrete) for _ in range (15))

#функция генерации дат
def generate_date():
  year = random.randint(2022, 2023)
  month = random.randint(1, 12)
  day = random.randint(1, 28)
  return datetime.date(year, month, day).strftime('%d.%m.%Y')


n= 10_000
data = {'user_id':[generate_user_id() for _ in range(n)],
        'order_id':np.random.randint(1,11,size = n),
        'click2delivery':np.random.normal(1440,200,size = n),
        'order_item_sum':np.random.exponential(100,size = n)+1,
        'retention':np.random.choice([1,2,3,4,5],size = n, p = [0.35,0.25,0.2,0.15,0.05]),
        'order_item_profit':np.round(np.random.normal(10, 1,size = n)),
        'sex':random.choices(['male','famele'], k=n),
        'city':random.choices(['New York', 'Los Angeles', 'Chicago', 'Houston', 'Philadelphia'],k=n),
        'date':[generate_date()for _ in range(n)]

        }

df = pd.DataFrame(data)
#конвертация даты в дату
df['date'] = pd.to_datetime(df['date']).dt.date
df.head()


<ipython-input-2-17142c60430c>:41: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['date'] = pd.to_datetime(df['date']).dt.date


,user_id,order_id,click2delivery,order_item_sum,retention,order_item_profit,sex,city,date
0,da211edc7bacj1a,7,1198.160258,2.022712,1,11.0,famele,Philadelphia,2023-10-19
1,j7182af742ej4dc,4,1451.348337,119.950380,1,9.0,famele,Houston,2023-09-05
2,6d552b8d6c35jcb,8,1462.691208,97.838195,1,10.0,male,Los Angeles,2022-10-26
3,13c1d6285f55j2c,5,1477.167526,104.558752,2,11.0,male,Philadelphia,2023-11-02
4,f1f86214b1b462c,7,1623.974303,47.901657,2,9.0,famele,Los Angeles,2023-07-01


In [3]:
#увеличение dataframe. в нашем df только 1 уникальный user id-один человек совершил только 1 покупку,что не совсем реалистично.
#создаем копию нашего дф
newdf1 = df[['user_id','sex','city']].copy ()
#дублируем инфу по пользователям
res1 = pd.DataFrame(np.repeat(newdf1.values,1, axis=0),columns=['user_id','sex','city']).reset_index(drop=True)
#обьединяем дф по юзерid
res2 = pd.concat([df, newdf1],keys=['user_id', 'user_id'], ignore_index=True).reset_index(drop=True)
#у нас образовались nan значения которые мы заполнил
res2['order_item_profit'] = res2['order_item_profit'].fillna((res2['order_item_profit'].mean()))
res2['order_item_sum'] = res2['order_item_sum'].fillna((res2['order_item_sum'].mean()))
res2['click2delivery'] = res2['click2delivery'].fillna((res2['click2delivery'].mean()))
res2['order_id'] = res2['order_id'].fillna(method='ffill')
res2['retention'] = res2['retention'].fillna(method='ffill')
res2['date'] = res2['date'].fillna(method='ffill')
print(res2)

               user_id  order_id  click2delivery  order_item_sum  retention  \
0      da211edc7bacj1a       7.0     1198.160258        2.022712        1.0   
1      j7182af742ej4dc       4.0     1451.348337      119.950380        1.0   
2      6d552b8d6c35jcb       8.0     1462.691208       97.838195        1.0   
3      13c1d6285f55j2c       5.0     1477.167526      104.558752        2.0   
4      f1f86214b1b462c       7.0     1623.974303       47.901657        2.0   
...                ...       ...             ...             ...        ...   
19995  8fjed23aeba94f7       5.0     1441.003580      100.756859        4.0   
19996  c3d4e72316b4cd2       5.0     1441.003580      100.756859        4.0   
19997  35a2e93b33bf5j3       5.0     1441.003580      100.756859        4.0   
19998  7b8c2769bfd93e4       5.0     1441.003580      100.756859        4.0   
19999  a3cjj26j5ja5b57       5.0     1441.003580      100.756859        4.0   

       order_item_profit     sex          city     

In [ ]:
#тест на дубликаты
res2.drop_duplicates()

,user_id,order_id,click2delivery,order_item_sum,retention,order_item_profit,sex,city,date
0,da211edc7bacj1a,7.0,1198.160258,2.022712,1.0,11.0000,famele,Philadelphia,2023-10-19
1,j7182af742ej4dc,4.0,1451.348337,119.950380,1.0,9.0000,famele,Houston,2023-09-05
2,6d552b8d6c35jcb,8.0,1462.691208,97.838195,1.0,10.0000,male,Los Angeles,2022-10-26
3,13c1d6285f55j2c,5.0,1477.167526,104.558752,2.0,11.0000,male,Philadelphia,2023-11-02
4,f1f86214b1b462c,7.0,1623.974303,47.901657,2.0,9.0000,famele,Los Angeles,2023-07-01
...,...,...,...,...,...,...,...,...,...
19995,8fjed23aeba94f7,5.0,1441.003580,100.756859,4.0,9.9949,famele,New York,2022-11-12
19996,c3d4e72316b4cd2,5.0,1441.003580,100.756859,4.0,9.9949,famele,Houston,2022-11-12
19997,35a2e93b33bf5j3,5.0,1441.003580,100.756859,4.0,9.9949,famele,New York,2022-11-12
19998,7b8c2769bfd93e4,5.0,1441.003580,100.756859,4.0,9.9949,famele,Los Angeles,2022-11-12


In [ ]:
res2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user_id            20000 non-null  object 
 1   order_id           20000 non-null  float64
 2   click2delivery     20000 non-null  float64
 3   order_item_sum     20000 non-null  float64
 4   retention          20000 non-null  float64
 5   order_item_profit  20000 non-null  float64
 6   sex                20000 non-null  object 
 7   city               20000 non-null  object 
 8   date               20000 non-null  object 
dtypes: float64(5), object(4)
memory usage: 1.4+ MB


In [4]:
#зальем в бд
con = sqlite3.connect('db')
res2.to_sql('dataset',con,index=False,if_exists='replace')

20000

In [5]:
def select(sql):
  return pd.read_sql(sql,con)

In [ ]:
sql = '''SELECT user_id,sex,city,date
FROM dataset
where user_id = 'da211edc7bacj1a'
'''

In [ ]:
select(sql)

,user_id,sex,city,date
0,da211edc7bacj1a,famele,Philadelphia,2023-10-19
1,da211edc7bacj1a,famele,Philadelphia,2022-11-12


In [ ]:
#проверка NaN в таблице SQL
sql = '''PRAGMA table_info(dataset);'''

In [ ]:
select(sql)

,cid,name,type,notnull,dflt_value,pk
0,0,user_id,TEXT,0,None,0
1,1,order_id,REAL,0,None,0
2,2,click2delivery,REAL,0,None,0
3,3,order_item_sum,REAL,0,None,0
4,4,retention,REAL,0,None,0
5,5,order_item_profit,REAL,0,None,0
6,6,sex,TEXT,0,None,0
7,7,city,TEXT,0,None,0
8,8,date,DATE,0,None,0


In [ ]:
#подсчет NAN в отдельных столбцах
sql = '''SELECT
sum(case when t.order_item_sum is NULL then 1 else 0 end) as is_null
FROM dataset t
'''

In [ ]:
select(sql)

,is_null
0,0


In [ ]:
#проверка дублей. Суть данного способа в агрегировании например 3 столбцов и искать совпадения
sql = '''SELECT
user_id,
order_id,
click2delivery,
COUNT(*) AS "Count"
FROM dataset
GROUP BY
user_id,
order_id,
click2delivery
HAVING COUNT(*) > 1;'''

In [ ]:
select(sql)

,user_id,order_id,click2delivery,Count


#EDA

*Разведочный анализ данных (Exploratory Data Analysis) – предварительное исследование Датасета (Dataset) с целью определения его основных характеристик, взаимосвязей между признаками, а также сужения набора методов, используемых для создания Модели (Model) Машинного обучения (Machine Learning).*

In [ ]:
#ежемесячный отчет с проверкой даты
#в 1 запросе мы генерируем список дат исходя из наших данных.У нас данные только по активным дня,но нет полных данных когда пользователь не активен.
#в 2 запросе проводим подсчет операций и их сумму
#в 3 запросе мы джоиним 2 запроса используя coalesce(замену пропусков на 0)
sql = '''select*from dataset t '''
min = '''select date(min(t.date),'start of month') from dataset t'''
max = '''select date(max(t.date),'start of month') from dataset t'''

sql = f'''WITH dates(month) AS (
  VALUES(({min}))
  UNION ALL
  SELECT date(month, '+1 month')
  FROM dates
  WHERE month < ({max})
),
trans_month as (select
date(t.date, 'start of month') as month,
count(1) as transaction_cnt,
sum(t.order_item_sum) as amount_sum

from dataset t
group by 1
order by 1)


SELECT t.month,
round(coalesce(tm.transaction_cnt,0),3) as transaction_cnt,
round(coalesce(tm.amount_sum,0),3) as amount_sum

FROM dates t
left join trans_month tm on t.month = tm.month
order by t.month'''

In [ ]:
select(sql)

,month,transaction_cnt,amount_sum
0,2022-01-01,389.0,39444.257
1,2022-02-01,417.0,42590.620
2,2022-03-01,442.0,42777.569
3,2022-04-01,427.0,42527.057
4,2022-05-01,416.0,42606.797
5,2022-06-01,424.0,40386.911
6,2022-07-01,385.0,36677.957
7,2022-08-01,387.0,40289.033
8,2022-09-01,416.0,43497.968
9,2022-10-01,412.0,39935.649


  **Res :** *Видим что количество транзакций и сумма примерно одинакова каждый месяц только 2022-11-01 выбивается из общей массы,это следует изучить подробнее*

In [ ]:
#сохраним в отдельную базу
dwithdate = select(sql)
dwithdate.to_sql('dwithdate',con,index=False,if_exists='replace')

24

In [ ]:
#нарастающий итог джоин таблицы саму на себя
sql = '''SELECT
t.month,ROUND(t.amount_sum)as amount_sum,ROUND(sum(r.amount_sum)) as cumsum
from dwithdate t
join dwithdate r on r.month <= t.month
group by t.month,ROUND(t.amount_sum)
'''
#нарастающий итог с оконной функцией
#sql = '''SELECT
#t.month,ROUND(t.amount_sum)as amount_sum,sum(amount_sum) over w as cumsum
#from dwithdate t
#window w as (
#  order by month
#  rows between unbounded preceding and current row
#)
#order by month;
#'''

In [ ]:
select(sql)

,month,amount_sum,cumsum
0,2022-01-01,39444.0,39444.0
1,2022-02-01,42591.0,82035.0
2,2022-03-01,42778.0,124812.0
3,2022-04-01,42527.0,167340.0
4,2022-05-01,42607.0,209946.0
5,2022-06-01,40387.0,250333.0
6,2022-07-01,36678.0,287011.0
7,2022-08-01,40289.0,327300.0
8,2022-09-01,43498.0,370798.0
9,2022-10-01,39936.0,410734.0


*проверка*


In [ ]:
sql = '''SELECT
ROUND(sum(t.order_item_sum)) as sum
FROM dataset t
'''

In [ ]:
select(sql)

,sum
0,2015137.0


In [ ]:
t=select(sql)

In [ ]:
t['amount_sum'].sum()

2015139.0

In [ ]:
#сводная таблица по категориальным переменным sex/order_id/city
sql = '''SELECT city,
count(1),
avg(order_item_profit) as avg_profit,
avg(order_item_sum) as avg_item_sum,
avg(click2delivery) as avg_click2delivery,
avg(retention) as avg_retention

FROM dataset
group by city
'''

In [ ]:
select(sql)

,city,count(1),avg_profit,avg_item_sum,avg_click2delivery,avg_retention
0,Chicago,3972,9.988387,99.763799,1440.699976,3.151309
1,Houston,3956,10.007056,100.718883,1439.164696,3.140799
2,Los Angeles,3848,9.988614,100.887865,1435.917348,3.119802
3,New York,4002,9.999199,101.127071,1446.642117,3.161169
4,Philadelphia,4222,9.991292,101.256377,1442.303187,3.160351


 **Res :** *по полу не обнаружено видимых отклоений,по полю город можно отметить что Philadelphia лидирует по количеству order*

In [ ]:
#мы уже знаем что средняя выручка на заказ 100 ед а прибыль 9-10.
#Попробуем узнать сколько клиентов по категориальным переменным пол/город ниже среднего а сколько выше
#меняя переменную в селекте t.city на t.sex можно оценить результаты
sql = '''SELECT t.city,
case when t.order_item_sum > 100 then '1.> 100'
when t.order_item_sum < 100 then '2.< 100'
else 'other' end as order_item_sum_bin,
count(1)
FROM dataset t
group by 1,2
'''

In [ ]:
select(sql)

,city,order_item_sum_bin,count(1)
0,Chicago,1.> 100,2707
1,Chicago,2.< 100,1265
2,Houston,1.> 100,2715
3,Houston,2.< 100,1241
4,Los Angeles,1.> 100,2637
5,Los Angeles,2.< 100,1211
6,New York,1.> 100,2778
7,New York,2.< 100,1224
8,Philadelphia,1.> 100,2897
9,Philadelphia,2.< 100,1325


*Видим что у нас большая часть стоимости заказов выше среднего.Такое возможно когда имееются не нормальное распределение*

In [ ]:
#топ3 заказа по выручке в каждом городе
sql = '''

with order_by_city_rnk as (SELECT t.*,

rank() over(partition by t.city order by t.order_item_sum desc) as rnk

FROM dataset t)

select user_id,city,rnk,order_item_sum
from order_by_city_rnk
where order_by_city_rnk.rnk <= 3

'''

In [ ]:
select(sql)

,user_id,city,rnk,order_item_sum
0,9d37f91ad29c597,Chicago,1,923.015850
1,971ad3fcd99j224,Chicago,2,810.719866
2,3839abae7863379,Chicago,3,798.576743
3,aj68743513eaaf8,Houston,1,823.483816
4,jac334af5e21de5,Houston,2,783.000967
5,e23982425b68c23,Houston,3,762.865035
6,b919915eda626aj,Los Angeles,1,672.949378
7,fc3f85de4ab9db9,Los Angeles,2,645.841054
8,57d96ce667d6f88,Los Angeles,3,620.319672
9,5115687aa2f9b17,New York,1,751.654363


*Можно отметить что Chicago выдялется более высокой суммой заказов и топовым покупателям можно делать специальные предложения*

In [ ]:
#список user_id клиентов которые приносят 20% выручки
#Функция ntile в основном используется для нарезки
sql = '''
with prepdat as (SELECT t.user_id,order_item_sum,
ntile(5) over(order by order_item_sum desc) as level2
from dataset t)

SELECT  t.user_id, t.order_item_sum
from prepdat t
where t.level2 = 1

'''

In [ ]:
select(sql)

,user_id,order_item_sum
0,9d37f91ad29c597,923.015850
1,aj68743513eaaf8,823.483816
2,971ad3fcd99j224,810.719866
3,3839abae7863379,798.576743
4,jac334af5e21de5,783.000967
...,...,...
3995,bd293j5bjd174af,100.756859
3996,2d61ab9e5c1f9c4,100.756859
3997,bb5ede91458ja65,100.756859
3998,e2b9839jf5482fc,100.756859


In [ ]:
#сводная таблица количественных item/profit/item_sum/click2delivery/retention переменных с группировкой по на начало месяца
sql = '''SELECT date(t.date,'start of month') as month,
count(1) as item_count,
round(sum(t.order_item_sum),3) as sum_by_month,
sum(t.order_item_profit) as profit_by_month,
avg(t.click2delivery) as avg_click2delivery,
avg(t.retention) as avg_retention
FROM dataset t
group by date(t.date,'start of month')
order by date(t.date,'start of month')
'''

In [ ]:
select(sql)

,month,item_count,sum_by_month,profit_by_month,avg_click2delivery,avg_retention
0,2022-01-01,389,39444.257,3873.0,1438.209059,2.339332
1,2022-02-01,417,42590.620,4143.0,1428.167380,2.342926
2,2022-03-01,442,42777.569,4434.0,1433.463063,2.242081
3,2022-04-01,427,42527.057,4279.0,1435.178014,2.274005
4,2022-05-01,416,42606.797,4166.0,1446.200001,2.295673
5,2022-06-01,424,40386.911,4235.0,1435.684049,2.351415
6,2022-07-01,385,36677.957,3826.0,1446.120792,2.405195
7,2022-08-01,387,40289.033,3868.0,1439.010550,2.248062
8,2022-09-01,416,43497.968,4148.0,1442.948073,2.206731
9,2022-10-01,412,39935.649,4133.0,1457.986904,2.235437


In [29]:
#cкользаящая средняя (квартальная 3 мес.) и разница между текущим
sql = '''
with base as(
SELECT
date(t.date,'start of month') as month,
count(1) as item_count,
round(sum(t.order_item_sum),3) as sum_by_month,
sum(t.order_item_profit) as profit_by_month

FROM dataset t
group by date(t.date,'start of month'))

Select
month,
sum_by_month,
avg(sum_by_month) over w as avg_sum,
(sum_by_month/avg(sum_by_month) over w)*100  as diff_sum,
profit_by_month,
avg(profit_by_month) over w as avg_profit,
(profit_by_month/avg(profit_by_month) over w)*100  as diff_profit
from base
window w as (order by month
rows between 1 preceding and 1 following
)

'''

In [30]:
select(sql)

,month,sum_by_month,avg_sum,diff_sum,profit_by_month,avg_profit,diff_profit
0,2022-01-01,39444.257,41017.438500,96.164603,3873.0,4008.000000,96.631737
1,2022-02-01,42590.620,41604.148667,102.371089,4143.0,4150.000000,99.831325
2,2022-03-01,42777.569,42631.748667,100.342046,4434.0,4285.333333,103.469197
3,2022-04-01,42527.057,42637.141000,99.741812,4279.0,4293.000000,99.673888
4,2022-05-01,42606.797,41840.255000,101.832068,4166.0,4226.666667,98.564669
5,2022-06-01,40386.911,39890.555000,101.244295,4235.0,4075.666667,103.909381
6,2022-07-01,36677.957,39117.967000,93.762431,3826.0,3976.333333,96.219298
7,2022-08-01,40289.033,40154.986000,100.333824,3868.0,3947.333333,97.990204
8,2022-09-01,43497.968,41240.883333,105.472930,4148.0,4049.666667,102.428183
9,2022-10-01,39935.649,377140.010667,10.589078,4133.0,37385.333333,11.055137


In [24]:
#оценка загрузки отделений.
#Будем счет считать что order_id это id отделения.Их у нас 10.
#И нам нужно посмотреть задержки в разрезе отделений.Разницу во времени поступающих заказов
#Будем считать что разница между новыми заказами с одного склада не должна привышать 1 день.
#Если привышает то штрафной бал.
#Аналогично можно посчитать например для click2delivery


sql = '''
with prepdat as (
SELECT t.date,
user_id,
order_id,
LAG(t.date) over(partition by order_id order by t.date ) as pred_dt ,
round((julianday(t.date) - julianday(lag(t.date) over (partition by t.order_id order by t.date))) ) as dt_diff,
case when round((julianday(t.date) - julianday(lag(t.date) over (partition by t.order_id order by t.date)))) > 1.0 then 1 else 0 end as signal
FROM dataset t)

SELECT t.order_id, sum(t.signal) as cnt_loss
FROM prepdat t
group by 1
order by 2 desc
'''

In [25]:
select(sql)

,order_id,cnt_loss
0,6.0,135
1,9.0,133
2,4.0,132
3,5.0,130
4,2.0,130
5,10.0,126
6,3.0,126
7,8.0,124
8,1.0,124
9,7.0,122


#XYZ

В результате XYZ-анализа объекты делят тоже на три группы:

    Группа X. В неё входят объекты с коэффициентом вариативности 0–10%. Это значит, что спрос не меняется более, чем на 10% в месяц.
    Группа Y. В неё входят объекты с коэффициентом вариативности 10–25%.
    Группа Z. В неё входят товары с самым непредсказуемым спросом — объекты с коэффициентом вариативности больше 25%.

XYZ-анализ чаще всего используют для анализа ассортимента, чтобы выделить наиболее ходовые товары, отказаться от неликвидных и оптимизировать складские помещения. Внашем примере это поле order id-будем считать фиалилал бизнеса

In [ ]:
#XYZ анализ
#1 подзапрос группировка данных
#2 подзапрос группировка по месяцам и годам
#3 подзапрос вычисление ковариации и отнесения к группам

sql = '''

with agg as (select order_id as tochka,
		sum(order_item_profit) as profit,
		sum(order_item_sum) as revenue
from dataset
group by order_id),


xyz as (select  STRFTIME('%m/%Y', date)
 as dat, order_id as tochka, sum(order_item_sum) as amount
from dataset
group by  STRFTIME('%m/%Y', date), order_id),



xyz_sales as
(select tochka,
case
	when coalesce(AVG(amount*amount) - AVG(amount)*AVG(amount)/avg(amount),0) <=0.1 then 'X'
	when coalesce(AVG(amount*amount) - AVG(amount)*AVG(amount)/avg(amount),0) <=0.25 then 'Y'
	when coalesce(AVG(amount*amount) - AVG(amount)*AVG(amount)/avg(amount),0) > 0.25 then 'Z'
  end as xyz_sales
from xyz
group by tochka
having count(distinct dat) >= 4)


SELECT * FROM xyz_sales
'''

In [ ]:
select(sql)

,tochka,xyz_sales
0,1.0,Z
1,2.0,Z
2,3.0,Z
3,4.0,Z
4,5.0,Z
5,6.0,Z
6,7.0,Z
7,8.0,Z
8,9.0,Z
9,10.0,Z


 **Res :** *видим что большинство точек имееют Z отметку-Чем сильнее «скачут» суммы продаж, тем выше коэффициент вариативности и тем ниже значение XYZ. В данном случае можно предположить что наша клиенская база нестабильна и бизнес юниты подвержены какими колебаниям например сезонные факторы или сезеонный характер дейятельности.Лучше использовать стратегию работы по предзаказу*

#ABC

В результате ABC-анализа объекты делят на три группы:

    Группа A. Это самые ценные позиции: 20% товаров, приносящих 80% прибыли.
    Группа B. Промежуточные позиции — 30% товаров, приносящие 15% прибыли.
    Группа C. Наименее ценные позиции. Это 80% товаров, которые приносят 5% прибыли.

In [ ]:
#ABC
#1 подзапрос группировка
#2 подзапрос отнесение к группам

sql = '''

with sales as (
SELECT order_id,
sum(order_item_profit) as order_item_profit
FROM dataset
group by order_id
)

SELECT order_id,
order_item_profit,
case when sum(order_item_profit) over (order by order_item_profit desc )/sum(order_item_profit) over () <=0.8 then 'A'
when sum(order_item_profit) over (order by order_item_profit desc )/sum(order_item_profit) over () <=0.95 then 'B'
else 'C'
end abc_group
FROM sales
order by order_item_profit desc

'''

In [ ]:
select(sql)

,order_id,order_item_profit,abc_group
0,5.0,109544.0,A
1,1.0,10546.0,A
2,10.0,10315.0,A
3,6.0,10234.0,A
4,7.0,10184.0,A
5,3.0,9956.0,B
6,9.0,9949.0,B
7,2.0,9831.0,B
8,4.0,9714.0,C
9,8.0,9625.0,C


 **Res :** * Для категории A дополнительно разивают. Категория B работают в таком количестве, чтобы покрыть спрос.Категория C прекращают работу. Другая стратегия — уменьшить их количество.*

#RFM

*RFM (РФМ) анализ — это метод сегментации клиентов компании, позволяющий делить их на группы в зависимости от суммы покупок и их регулярности. Нужен для того, чтобы продумать схемы работы с каждой группой клиентов. Аббревиатура RFM расшифровывается как Recency, Frequency и Monetary*

In [ ]:
#RFM
#в 1 подзапросе группируем данные и получаем данные для rfm
#в 2 подзапросе делим с помощью ntile делим на 4 группы(по стандарту 3 группы но в данном случае шаблон на 4)
#в 3 подзапросе переводим число в строку и склеиваем значения rfm
#в 4 подзапросе соотносим к категориям

sql = '''
    WITH rfm AS (
    SELECT
    order_id,
    SUM(order_item_profit) AS MonetaryValue,
    AVG(order_item_profit) AS AvgMonetaryValue,
    COUNT(order_id) AS Frequency,
    JULIANDAY(MAX(date) over()) -JULIANDAY(MAX(date)) AS Recency

    FROM dataset
    GROUP BY order_id),

    rfm_calc AS (
    SELECT R.*,
          NTILE(4) OVER (ORDER BY Recency) rfm_recency,
          NTILE(4) OVER (ORDER BY Frequency) rfm_frequency,
          NTILE(4) OVER (ORDER BY MonetaryValue) rfm_monetary
    FROM rfm R
    ORDER BY 4 DESC
  ),

complite as (SELECT
  C.*,
rfm_recency || rfm_frequency || rfm_monetary AS rfm_cell_string
FROM rfm_calc C)


SELECT  rfm_recency, rfm_monetary, rfm_cell_string,
  CASE
   WHEN cast(rfm_cell_string AS INT) IN (111, 112, 121, 122, 123, 132, 211, 212, 114, 141) THEN 'lost_customers'
   WHEN cast(rfm_cell_string AS INT) IN (133, 134, 143, 244, 334, 343, 344) THEN 'slipping away, cannot lose'
   WHEN cast(rfm_cell_string AS INT) IN (311, 411, 331) THEN 'new_customers'
   WHEN cast(rfm_cell_string AS INT) IN (222, 223, 233, 322) THEN 'potential churners'
   WHEN cast(rfm_cell_string AS INT) IN (323, 333, 321, 422, 332, 432) THEN 'active'
   WHEN cast(rfm_cell_string AS INT) IN (433, 434, 443, 444) THEN 'loyal'
   ELSE NULL END AS type

FROM complite

order by rfm_cell_string desc

'''

In [ ]:
select(sql)

,rfm_recency,rfm_monetary,rfm_cell_string,type
0,4,4,444,loyal
1,4,1,411,new_customers
2,3,4,344,"slipping away, cannot lose"
3,3,1,311,new_customers
4,2,3,233,potential churners
5,2,3,233,potential churners
6,2,2,222,potential churners
7,1,2,122,lost_customers
8,1,2,122,lost_customers
9,1,1,111,lost_customers


 **Res :**  *Исходя из типа можно предложить следующую стратегию:

  loyal - увеличить чек и цель заставить пользоваться чаще
  
  new_customers- улучшения узнаваемости бренда,скидки на крупные заказы,
  
  slipping away- восстановление лояльности,персональные акции
  
  potential churners- ограниченные по времени предложения
  
  lost_customers-информация о револентных предложениях*

#Когортный анализ

*Когортный анализ — способ изучить группы пользователей и клиентов, объединённых общими признаками в определённый период времени*

In [27]:
#Когортный анализ LTV
#1 подзапрос собираем данные,когорты как данные за мес.дифф разница между первой покупкой и другими покупками
#2 создаем когорты

sql = '''

with a as (
select
user_id,
date,
first_value(STRFTIME('%m/%Y', date)) over (partition by user_id order by date) as cohort,
JULIANDAY(date) - first_value(JULIANDAY(date)) over(partition by user_id order by date) as diff,
order_item_sum
from dataset b


)

select
        cohort,
        count(DISTINCT user_id) as cnt,
        max(diff) as max_diff,
        sum(case when diff = 0 then order_item_sum end) / count(distinct user_id) as "0",
        case
                when max(diff) >= 30 then sum(case when diff <= 30 then order_item_sum end) / count( distinct user_id)
        end as "30",
        case
                when max(diff) >= 90 then sum(case when diff <= 90 then order_item_sum end) / count( distinct user_id)
        end as "90",
          case
                when max(diff) >= 120 then sum(case when diff <= 120 then order_item_sum end) / count( distinct user_id)
        end as "90",
          case
                when max(diff) >= 150 then sum(case when diff <= 150 then order_item_sum end) / count( distinct user_id)
        end as "90",
        case
                when max(diff) >= 180 then sum(case when diff <= 180 then order_item_sum end) / count( distinct user_id)
        end as "180",
        case
                when max(diff) >= 360 then sum(case when diff <= 360 then order_item_sum end) / count( distinct user_id)
        end as "360",
        case
                when max(diff) >= 400 then sum(case when diff <= 400 then order_item_sum end) / count( distinct user_id)
        end as "400"

from a
group by cohort
order by cohort


'''

In [28]:
select(sql)

,cohort,cnt,max_diff,0,30,90,90,90,180,360,400
0,01/2022,389,315.0,101.399118,101.399118,101.399118,101.399118,101.399118,101.399118,NaN,NaN
1,02/2022,417,284.0,102.135779,102.135779,102.135779,102.135779,102.135779,102.135779,NaN,NaN
2,03/2022,442,256.0,96.781829,96.781829,96.781829,96.781829,96.781829,96.781829,NaN,NaN
3,04/2022,427,225.0,99.594982,99.594982,99.594982,99.594982,99.594982,99.594982,NaN,NaN
4,05/2022,416,195.0,102.420185,102.420185,102.420185,102.420185,102.420185,151.345391,NaN,NaN
5,06/2022,424,164.0,95.252149,95.252149,95.252149,95.252149,146.105847,NaN,NaN,NaN
6,07/2022,385,134.0,95.267422,95.267422,95.267422,142.112818,NaN,NaN,NaN,NaN
7,08/2022,387,103.0,104.106027,104.106027,159.040646,NaN,NaN,NaN,NaN,NaN
8,09/2022,416,72.0,104.562424,104.562424,NaN,NaN,NaN,NaN,NaN,NaN
9,10/2022,412,42.0,96.931187,156.113614,NaN,NaN,NaN,NaN,NaN,NaN


 **Res :** *Видим что в основном когорты у нас не растущие пользователи совершили одну покупку и далее не совершают.Кроме когорты 11.2022 когда был всплеск покупок,что также говорит о сезонном характере бизнеса и рекомендациях работать по предзаказу*